The Specimen was sent to the lab for HLA-A typing by full gene sequencing. Two sequences were acquired. Both sequences are put into a single fasta file.

In [1]:
import json
from Bio import SeqIO # biopython
from yattag import Doc # for generating HTML
from fhirclient import client
import fhirclient.models.codeableconcept as cc
import fhirclient.models.coding as co
import fhirclient.models.sequence as seq
import fhirclient.models.narrative as nar

In [2]:
# check the fasta file for ids
myFastas = list(SeqIO.parse("seq.fasta", "fasta"))
for s in myFastas:
    print(s.id)

HLA-A*01:01:01:01
HLA-A*01:02


In [3]:
mySeq = seq.Sequence()
for eproperty in mySeq.elementProperties():
    print(eproperty)

('id', 'id', <class 'str'>, False, None, False)
('implicitRules', 'implicitRules', <class 'str'>, False, None, False)
('language', 'language', <class 'str'>, False, None, False)
('meta', 'meta', <class 'fhirclient.models.meta.Meta'>, False, None, False)
('contained', 'contained', <class 'fhirclient.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('text', 'text', <class 'fhirclient.models.narrative.Narrative'>, False, None, False)
('coordinateSystem', 'coordinateSystem', <class 'int'>, False, None, True)
('device', 'device', <class 'fhirclient.models.fhirreference.FHIRReference'>, False, None, False)
('identifier', 'identifier', <class 'fhirclient.models.identifier.Identifier'>, True, None, False)
('observedSeq', 'observedSeq', <class 'str'>, False, None, False)
('patient', 'patient', 

In [4]:
for eproperty in mySeq.elementProperties():
    if eproperty[5] == True:
        print(eproperty)

('coordinateSystem', 'coordinateSystem', <class 'int'>, False, None, True)


In [5]:
mySeq.coordinateSystem = 0
mySeq.type = 'dna'

In [6]:
seqRef = seq.SequenceReferenceSeq()
for eproperty in seqRef.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('chromosome', 'chromosome', <class 'fhirclient.models.codeableconcept.CodeableConcept'>, False, None, False)
('genomeBuild', 'genomeBuild', <class 'str'>, False, None, False)
('referenceSeqId', 'referenceSeqId', <class 'fhirclient.models.codeableconcept.CodeableConcept'>, False, None, False)
('referenceSeqPointer', 'referenceSeqPointer', <class 'fhirclient.models.fhirreference.FHIRReference'>, False, None, False)
('referenceSeqString', 'referenceSeqString', <class 'str'>, False, None, False)
('strand', 'strand', <class 'int'>, False, None, False)
('windowEnd', 'windowEnd', <class 'int'>, False, None, True)
('windowStart', 'windowStart', <class 'int'>, False, None, True)


In [7]:
refSeqId = cc.CodeableConcept()
refSeqIdCoding = co.Coding()
refSeqIdCoding.system = 'http://www.ebi.ac.uk/ipd/imgt/hla/'
refSeqIdCoding.version = '3.23'
refSeqIdCoding.code = 'HLA00001'
refSeqIdCoding.display = myFastas[0].id
refSeqId.coding = [refSeqIdCoding] # is_list for coding is True, so must be put into a list
print(json.dumps(refSeqId.as_json(), indent=4))

{
    "coding": [
        {
            "code": "HLA00001",
            "display": "HLA-A*01:01:01:01",
            "system": "http://www.ebi.ac.uk/ipd/imgt/hla/",
            "version": "3.23"
        }
    ]
}


In [8]:
seqRef.referenceSeqId = refSeqId
seqRef.windowStart = 0
seqRef.windowEnd = len(myFastas[0].seq) # interval coordinate system, whole sequence
seqRef.referenceSeqString = str(myFastas[0].seq)
mySeq.referenceSeq = seqRef
# print(json.dumps(mySeq.as_json(), indent=4))
print(myFastas[0].id)
doc, tag, text = Doc().tagtext()
with tag('div', xmlns="http://www.w3.org/1999/xhtml"):
    with tag('pre'):
        text('this is ' +  
             myFastas[0].id + 
             ' end'
        )
# print(doc.getvalue())


HLA-A*01:01:01:01


In [9]:
myText = nar.Narrative()
myText.status = 'generated' # status is required
# use Doc from the yattag package to build the div
# use the id from the fasta file for the text
myDiv, tag, text = Doc().tagtext()
with tag('div', xmlns='http://www.w3.org/1999/xhtml'):
    with tag('table'):
        with tag('tr'):
            with tag('td'):
                text('fasta id')
            with tag('td'):
                text(myFastas[0].id)
        with tag('tr'):
            with tag('td'):
                text('reference display') 
            with tag('td'):
                text(mySeq.referenceSeq.referenceSeqId.coding[0].display)
        with tag('tr'):
            with tag('td'):
                text('reference system')
            with tag('td'):
                text(mySeq.referenceSeq.referenceSeqId.coding[0].system)
        with tag('tr'):
            with tag('td'):
                text('reference code') 
            with tag('td'):
                text(mySeq.referenceSeq.referenceSeqId.coding[0].code)
        with tag('tr'):
            with tag('td'):        
                text('coordinate system') 
            with tag('td'):
                text(mySeq.coordinateSystem)
        with tag('tr'):
            with tag('td'):        
                text('start =') 
            with tag('td'):
                text(mySeq.referenceSeq.windowStart)
        with tag('tr'):
            with tag('td'):        
                text('end =') 
            with tag('td'):
                text(mySeq.referenceSeq.windowEnd)
# print(doc.getvalue())
myText.div = myDiv.getvalue()
mySeq.text = myText
print(json.dumps(mySeq.as_json(), indent=4))

{
    "text": {
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><table><tr><td>fasta id</td><td>HLA-A*01:01:01:01</td></tr><tr><td>reference display</td><td>HLA-A*01:01:01:01</td></tr><tr><td>reference system</td><td>http://www.ebi.ac.uk/ipd/imgt/hla/</td></tr><tr><td>reference code</td><td>HLA00001</td></tr><tr><td>coordinate system</td><td>0</td></tr><tr><td>start =</td><td>0</td></tr><tr><td>end =</td><td>3503</td></tr></table></div>",
        "status": "generated"
    },
    "coordinateSystem": 0,
    "referenceSeq": {
        "referenceSeqId": {
            "coding": [
                {
                    "code": "HLA00001",
                    "display": "HLA-A*01:01:01:01",
                    "system": "http://www.ebi.ac.uk/ipd/imgt/hla/",
                    "version": "3.23"
                }
            ]
        },
        "referenceSeqString": "CAGGAGCAGAGGGGTCAGGGCGAAGTCCCAGGGCCCCAGGCGTGGCTCTCAGGGTCTCAGGCCCCGAAGGCGGTGTATGGATTGGGGAGTCCCAGCCTTGGGGATTCCCCAACTCCG

In [10]:
# write resource to file
fname = 'sequence.json'
with open(fname, 'w') as outfile:
    json.dump(mySeq.as_json(), outfile, indent=4)
print("patient json written to file {fn}".format(fn=fname))

patient json written to file sequence.json


In [11]:
doit = True
if doit:
    settings = {
        'app_id': 'my_web_app',
        'api_base': 'http://fhirtest.b12x.org/baseDstu3'
    }
    smart = client.FHIRClient(settings=settings)
    response = mySeq.create(smart.server)
    print(json.dumps(response.json(), indent=4))
    print(response.headers['location'])

{
    "resourceType": "OperationOutcome",
    "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><h1>Operation Outcome</h1><table border=\"0\"><tr><td style=\"font-weight: bold;\">INFORMATION</td><td>[]</td><td><pre>Successfully created resource &quot;Sequence/2613/_history/1&quot; in 3ms</pre></td>\n\t\t\t\t\t\n\t\t\t\t\n\t\t\t</tr>\n\t\t</table>\n\t</div>"
    },
    "issue": [
        {
            "severity": "information",
            "code": "informational",
            "diagnostics": "Successfully created resource \"Sequence/2613/_history/1\" in 3ms"
        }
    ]
}
http://fhirtest.b12x.org/baseDstu3/Sequence/2613/_history/1


```python
    import fhirclient.models.narrative as n
    narrative = n.Narrative()
    narrative.div = (
        '<div xmlns="http://www.w3.org/1999/xhtml">'
        '<pre>HLA-A*01:01:01:01, exon 3</pre>'
        '</div>'
    )
    narrative.status = 'generated'
    sequence.text = narrative
```


```xml
<text>
    <status value="generated"/>
    <div xmlns="http://www.w3.org/1999/xhtml">
        <pre>&quot;HLA-A*01:01:01:01, exon 2&quot;</pre>
    </div>
</text>
<type value="dna"/>
<coordinateSystem value="0"/>
<referenceSeq>
    <referenceSeqId>
        <coding>
            <system value="http://www.ebi.ac.uk/ipd/imgt/hla/"/>
            <version value="3.23"/>
            <code value="HLA00001"/>
            <display value="HLA-A*01:01:01:01"/>
        </coding>
    </referenceSeqId>
    <windowStart value="503"/>
    <windowEnd value="773"/>
</referenceSeq>
<observedSeq value="GCTCCCAC..."/>
```